In [ ]:
!pip install --upgrade --no-cache-dir gdown
!pip install diffusers
!pip install datasets
!pip install peft
!pip install gradio

In [ ]:
import gdown
url = "https://drive.google.com/uc?id=1mR_axgYLVaFzLJ3QZw8NZlTwk6WTpcXW"
output = "/content/code.zip"
gdown.download(url, output, quiet=False)

In [ ]:
import zipfile
import os

zip_file_path = '/content/code.zip'
extract_folder_path = '/content/'

# Check if the zip file exists
if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder_path)
else:
    print("The specified zip file does not exist.")


In [ ]:
import torch
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

In [ ]:
from models import Diffusion
d = Diffusion(device , "LMSD")

In [ ]:
from image_utils import imageToTensor

In [ ]:
from datasets import load_dataset
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class CustomCartoonBlipCaptionsDataset(Dataset):
    def __init__(self, split='train', transform=None):
        self.dataset = load_dataset("macadeliccc/simpsons-images", split=split)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        image = self.dataset[idx]["image"]
        caption = self.dataset[idx]["text"]
        if self.transform:
            image = image.convert('RGB')
            image = self.transform(image)
        return image, caption

transform = transforms.Compose([
         transforms.Resize(512, interpolation=transforms.InterpolationMode.BILINEAR),
          transforms.CenterCrop(512) ,
          transforms.ToTensor(),
          transforms.Normalize([0.5], [0.5]),
])


custom_dataset = CustomCartoonBlipCaptionsDataset(split='train', transform=transform)

sample_image, sample_caption = custom_dataset[0]
print(sample_image.shape)
print(sample_caption)

In [ ]:
d.train(custom_dataset  ,createPickle = True  , epochs = 50)

In [ ]:
trainable_params = {name: param for name, param in d.unet.model.named_parameters() if param.requires_grad}
torch.save(trainable_params, '/content/simpsons.pth')